# Text Pre Processor - Implantação

Este é um componente que utiliza a biblioteca [nltk](https://www.nltk.org/) e [ftfy](https://pypi.org/project/ftfy/) e [regex](https://docs.python.org/3/library/re.html) para pré processar textos que entrrão em outros componentes.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [31]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-text-pre-processor/pre_processor.py

--2020-10-23 12:40:32--  https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-text-pre-processor/pre_processor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.92.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.92.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5554 (5.4K) [text/plain]
Saving to: ‘pre_processor.py.3’

pre_processor.py.3  100%[===================>]   5.42K  --.-KB/s    in 0s      

2020-10-23 12:40:33 (36.2 MB/s) - ‘pre_processor.py.3’ saved [5554/5554]



In [35]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
from pre_processor import Preprocessor


class Model:
    def __init__(self):
        artifacts = joblib.load("/tmp/data/preprocessor.joblib")
        self.preprocessing_tasks = artifacts["preprocessing_tasks"]
        self.model_parameters = artifacts["model_parameters"]
        self.columns = artifacts["columns"]
         
    def class_names(self):
        return ['text']

    def predict(self, X, feature_names, meta=None):
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)
            data = df[self.columns]
        
        preprocessor = Preprocessor(self.preprocessing_tasks,self.model_parameters)
        texts = preprocessor.preprocess(X)
        result = []
        for text in texts:
            result.append([text])
        return np.array(result)

Overwriting Model.py


## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [36]:
%%writefile contract.json
{
    "features": [
        {
            "name": "review",
            "ftype": "categorical",
            "values": ["How are you doing?"]
        }
    ],
    "targets": []
}

Overwriting contract.json


In [37]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")

----------------------------------------
SENDING NEW REQUEST:

[['How are you doing?']]
RECEIVED RESPONSE:
meta {
}
data {
  names: "t:0"
  ndarray {
    values {
      list_value {
        values {
          string_value: "how are you doing"
        }
      }
    }
  }
}


2020-10-23 12:42:13.768416: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-10-23 12:42:13.768474: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-10-23 12:42:18,119 - seldon_core.microservice:main:190 - INFO:  Starting microservice.py:main
2020-10-23 12:42:18,122 - seldon_core.microservice:main:246 - INFO:  Parse JAEGER_EXTRA_TAGS []
2020-10-23 12:42:18,123 - seldon_core.microservice:main:254 - DEBUG:  Log level set to DEBUG:10
2020-10-23 12:42:18,123 - seldon_core.microser